# Ensembles

Test the result of using several ANN for the same prediction task


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split #pip install -U scikit-learn
from sklearn.preprocessing import MinMaxScaler

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

import time
import operator

Using TensorFlow backend.


In [2]:
datos = pd.read_csv('datos.csv')

datos.sample(frac=1)

x_data = datos.drop(labels=['PTO_X', 'PTO_Y'], axis=1)
y_data = datos.drop(['WI0', 'WI1', 'WI2', 'WI3', 'WI4', 'WI5', 'WI6', 'WI7', 'WI8', 'WI9'], axis=1)

mejores = {}
macs_old = ['WI0', 'WI1', 'WI2', 'WI3', 'WI4', 'WI5', 'WI6', 'WI7', 'WI8', 'WI9']
for i in macs_old:
    count = 0
    for j in x_data[i]:
        if(j == -99):
            count += 1
    mejores[i] = count
    #print(i, " --> ", count)
    
sorted(mejores.items(), key=operator.itemgetter(1))
    
#--> Probar con los que menos -99 tienen: WI0, WI1, WI3, WI4, WI5, WI7

[('WI0', 71),
 ('WI7', 190),
 ('WI1', 284),
 ('WI3', 285),
 ('WI5', 293),
 ('WI4', 294),
 ('WI6', 310),
 ('WI2', 330),
 ('WI9', 360),
 ('WI8', 1281)]

In [3]:
import winsound
def beep():
    duration = 500  # millisecond
    freq = 900  # Hz
    winsound.Beep(freq, duration)
    winsound.Beep(freq, duration)
    winsound.Beep(freq, 2*duration)

In [4]:
def calcError(real, pred):
    error = 0.
    real0 = np.asarray(real)
    for i in range(real.shape[0]):
        error = error + np.sqrt(pow(pred[i][0] - real0[i][0], 2) + pow(pred[i][1] - real0[i][1], 2))
    return error  /  real.shape[0]

In [5]:
#################################################################################################
#ANN Model
batch_size=128
epocas=60000

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(3, activation='relu', input_shape=(x_data.shape[1],), name='cesin'))
    model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dense(3, activation='relu'))
    model.add(layers.Dense(2, name='cesout'))
    model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss=losses.mse, metrics=[metrics.mae])
    return model

In [6]:
#################################################################################################
#Loading the data
datos = pd.read_csv('datos.csv')

datos.sample(frac=1)

#x_data = datos.drop(labels=['PTO_X', 'PTO_Y', 'WI3', 'WI5', 'WI8', 'WI9'], axis=1)
x_data = datos.drop(labels=['PTO_X', 'PTO_Y', 'WI2', 'WI6', 'WI8', 'WI9'], axis=1)

y_data = datos.drop(['WI0', 'WI1', 'WI2', 'WI3', 'WI4', 'WI5', 'WI6', 'WI7', 'WI8', 'WI9'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=101)

print("TRAIN DATA: ", x_train.shape)
print("TEST  DATA: ", x_test.shape)

TRAIN DATA:  (1579, 6)
TEST  DATA:  (678, 6)


In [7]:
#################################################################################################
#Normalizing the data
mean = x_train.mean(axis=0).mean()
std  = x_train.std(axis=0).std()
print("MEAN: %f,  STD DEV: %f" % (mean, std))
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

MEAN: -62.442791,  STD DEV: 2.601134


In [ ]:
#################################################################################################
# K Fold Study
k = 3
n_samples = x_data.size // x_data.shape[1] // k
modelos = []

print(time.ctime())
print('Samples by fold: ', n_samples, '\n')
all_scores = []
all_hist = []
for i in range(k):
    print('processing fold #', i)

    x_valid_k = x_data[i * n_samples : (i + 1) * n_samples]
    y_valid_k = y_data[i * n_samples : (i + 1) * n_samples]
    x_data_k = np.concatenate([x_data[ : i * n_samples], x_data[(i + 1) * n_samples : ]], axis=0)
    y_data_k = np.concatenate([y_data[ : i * n_samples], y_data[(i + 1) * n_samples : ]], axis=0)

    modelos.append(build_model())
    hist_k = modelos[i].fit(x_data_k, y_data_k, validation_data=(x_valid_k, y_valid_k), epochs=epocas, batch_size=batch_size, verbose=0)
    #all_hist.append(hist3.history['val_loss'])
    y_predicted = modelos[i].predict(x_test)
    all_scores.append(calcError(y_test, y_predicted))
    all_hist.append(hist_k3.history['val_mean_absolute_error'])
    print(time.ctime())
    print("  Min error %f at epoch %d" % (np.min(hist_k.history['val_loss'][:]), np.argmin(hist_k.history['val_loss'][:])))
    
    y_predicted = model[i].predict(x_test)
    print("  CES ERROR: ", calcError(y_test, y_predicted))
beep()

Wed Jan 24 13:35:19 2018
Samples by fold:  752 

processing fold # 0


In [ ]:
y_predicted_tot = model[0].predict(x_test)
for i in range(1..k):
    y_predicted_tot = model[i].predict(x_test)

print("  CES ERROR: ", calcError(y_test, y_predicted/k))